In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, momentum=0.9)

In [20]:
import torch.optim as optim
from torch.utils.data import DataLoader
import sys
sys.path.append('..')
from shared.models import *
from shared.datasets import *

bs = 16
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BaselineNet(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

device(type='cuda')

In [ ]:
mimic_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                            csv_path='./splits.csv', mode='base_train', resize=224)
train_loader = DataLoader(mimic_dataset, batch_size=bs, shuffle=True, num_workers=4)

for epoch in range(2):
    running_loss = 0.0
    for k, (data_inputs, data_labels) in enumerate(train_loader):
        
        inputs, labels = data_inputs.to(device), data_labels.to(device)
        # Set gradient to 0.
        optimizer.zero_grad()
        # Feed forward.
        pred = model(inputs)
        # Loss calculation.
        loss = criterion(pred, labels)
        # Gradient calculation.
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print(f'[{epoch+1}] loss: {running_loss/k}')
        

In [1]:
import torch.optim as optim
from torch.utils.data import DataLoader
import sys
sys.path.append('..')
from shared.models import *
from shared.datasets import *

bs = 64
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BaselineNet(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

mimic_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                            csv_path='./splits.csv', mode='base_train', resize=224)
train_loader = DataLoader(mimic_dataset, batch_size=bs, shuffle=True, num_workers=5)

for epoch in range(2):
    running_loss = 0.0
    for k, (data_inputs, data_labels) in enumerate(train_loader):
        
        inputs, labels = data_inputs.to(device), data_labels.to(device)
        # Set gradient to 0.
        optimizer.zero_grad()
        # Feed forward.
        pred = model(inputs)
        # Loss calculation.
        loss = criterion(pred, labels)
        # Gradient calculation.
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print(f'[{epoch+1}] loss: {running_loss/k}')

[1] loss: 2.0349741466583744
[2] loss: 1.9368053940034682


In [14]:
print(pred.size())

torch.Size([16, 10])


In [11]:
print(labels)

('Cardiomegaly', 'Support Devices', 'Lung Opacity', 'No Finding', 'Edema', 'No Finding', 'Cardiomegaly', 'Atelectasis', 'Edema', 'No Finding', 'No Finding', 'No Finding', 'Support Devices', 'Atelectasis', 'Support Devices', 'Support Devices')


In [9]:
test = {'Atelectasis': 0,
 'Cardiomegaly': 1,
 'Consolidation': 2,
 'Edema': 3,
 'Fracture': 4,
 'Lung Opacity': 5,
 'No Finding': 6,
 'Pneumonia': 7,
 'Pneumothorax': 8,
 'Support Devices': 9}

6

In [6]:
test['b'] = 2

In [7]:
list = ['bruh', 'b', 'b']

In [8]:
test[list]

TypeError: unhashable type: 'list'